<a href="https://colab.research.google.com/github/Carol-Coutinho/OU3/blob/main/AVS01_OU3_Caroline_Coutinho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AVS_01_OU3

Caroline Coutinho

Flash Isotérmico Binário

Mistura escolhida: Tolueno e Benzeno

Primeiro vamos preparar o Kernel:

In [ ]:
# Preparando kernel
import pandas as pd
#import pickle
!pip3 install pickle5
import pickle5 as pickl
import numpy as np

     |████████████████████████████████| 132 kB 5.4 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219323 sha256=0d4b4e5db9e2ea5b807363fb05f742472c7a6c0bdcf7ba0434d5342b461f2d36
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [ ]:
# Para ler/carregar
with open('/content/databank_properties.pickle', 'rb') as handle:
    dados, all_units = pickl.load(handle)

Analisando o banco de dados e buscando os componentes necessários: Tolueno e Benzeno.


In [ ]:
dados.shape


(468, 31)

In [ ]:
dados.columns

Index(['num', 'name', 'molar_mass', 'fusion_point', 'boiling_point',
       'critical_temp', 'critical_pressure', 'critical_volume', 'zc',
       'acentric_factor', 'liq_density', 'temp_ref_liq_density',
       'dipole_moment', 'cp_a', 'cp_b', 'cp_c', 'cp_d', 'visc_b', 'visc_c',
       'delta_h0f_gas', 'delta_g0f_gas', 'pvap_a', 'pvap_b', 'pvap_c',
       'temp_max_pvap', 'temp_min_pvap', 'pvap_h_a', 'pvap_h_b', 'pvap_h_c',
       'pvap_h_d', 'delta_h_vap_bp'],
      dtype='object')

In [ ]:
# Buscando o primeiro componente:
filtro1 = dados.name.str.contains('BENZENE')
dados[filtro1]


,num,name,molar_mass,fusion_point,boiling_point,critical_temp,critical_pressure,critical_volume,zc,acentric_factor,liq_density,temp_ref_liq_density,dipole_moment,cp_a,cp_b,cp_c,cp_d,visc_b,visc_c,delta_h0f_gas,delta_g0f_gas,pvap_a,pvap_b,pvap_c,temp_max_pvap,temp_min_pvap,pvap_h_a,pvap_h_b,pvap_h_c,pvap_h_d,delta_h_vap_bp
14,383,"1,2,3-TRIMETHYLBENZENE",120.195,247.7,449.2,664.5,34.1,430.0,0.270,0.390,0.894,293.0,0.6,-1.658000,0.151300,-0.000079,1.579000e-08,NaN,NaN,-2.29,29.77,16.2121,3670.22,-66.07,479.00,329.0,NaN,NaN,NaN,NaN,9570.0
16,384,"1,2,4-TRIMETHYLBENZENE",120.195,227.0,442.5,649.1,31.9,430.0,0.258,0.390,0.880,289.0,0.3,-1.115000,0.149000,-0.000078,1.523000e-08,872.74,297.75,-3.33,27.95,16.2190,3622.58,-64.59,471.00,324.0,56.241,-7256.56,-5.459,7.270,9380.0
24,385,"1,3,5-TRIMETHYLBENZENE",120.195,228.4,437.9,637.3,30.9,433.0,0.260,0.398,0.865,293.0,0.1,-4.679000,0.160600,-0.000088,1.839000e-08,437.52,268.27,3.84,28.19,16.2893,3614.19,-63.57,466.00,321.0,58.041,-7326.78,-5.706,7.220,9330.0
28,407,"1,4-DIETHYLBENZENE",134.222,231.0,456.9,657.9,27.7,480.0,0.250,0.403,0.862,293.0,0.1,-8.937000,0.207100,-0.000133,3.370000e-08,NaN,NaN,-5.32,32.95,16.1140,3657.22,-71.18,487.00,335.0,NaN,NaN,NaN,NaN,9410.0
45,380,1-METHYL-2-ETHYLBENZENE,120.195,192.3,438.3,651.0,30.0,460.0,0.260,0.294,0.881,293.0,0.0,-3.928000,0.167100,-0.000098,2.228000e-08,NaN,NaN,0.29,31.33,16.1253,3535.33,-65.85,467.00,321.0,64.337,-7662.94,-6.617,7.180,9290.0
46,404,1-METHYL-2-ISOPROPYLBENZENE,134.222,NaN,451.5,670.0,28.6,NaN,NaN,0.277,0.876,293.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,15.9809,3564.52,-70.00,481.00,330.0,NaN,NaN,NaN,NaN,0.0
47,405,1-METHYL-2-ISOPROPYLBENZENE,134.222,NaN,448.3,666.0,29.0,NaN,NaN,0.279,0.861,293.0,0.0,-11.646000,0.216500,-0.000145,3.887000e-08,NaN,NaN,-7.00,0.00,15.9811,3543.79,-69.22,478.00,328.0,67.726,-8033.58,-7.076,8.390,9110.0
48,381,1-METHYL-3-ETHYLBENZENE,120.195,177.6,434.5,637.0,28.0,490.0,0.260,0.360,0.865,293.0,0.0,-6.926000,0.174200,-0.000104,2.388000e+00,8.00,0.00,0.00,-0.46,30.2200,16.15,3521.08,-64.64,463.0,318.000,65.67,-7678.110,-6.815,7.2
49,382,1-METHYL-4-ETHYLBENZENE,120.195,210.8,435.2,640.0,29.0,470.0,0.260,0.322,0.861,293.0,0.0,-6.523000,0.171400,-0.000101,2.279000e-08,463.17,266.08,-0.49,30.28,16.1135,3516.31,-64.23,463.00,318.0,61.404,-7422.59,-6.212,7.230,9180.0
50,406,1-METHYL-4-ISOPROPYLBENZENE,134.222,200.0,450.3,653.0,27.9,NaN,NaN,0.371,0.857,293.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,15.9424,3539.21,-70.10,480.00,329.0,63.225,-7800.97,-6.432,8.410,0.0


In [ ]:
# Buscando o segundo componente: 
filtro2 = dados.name.str.contains('TOLUENE')
dados[filtro2]

,num,name,molar_mass,fusion_point,boiling_point,critical_temp,critical_pressure,critical_volume,zc,acentric_factor,liq_density,temp_ref_liq_density,dipole_moment,cp_a,cp_b,cp_c,cp_d,visc_b,visc_c,delta_h0f_gas,delta_g0f_gas,pvap_a,pvap_b,pvap_c,temp_max_pvap,temp_min_pvap,pvap_h_a,pvap_h_b,pvap_h_c,pvap_h_d,delta_h_vap_bp
440,286,TOLUENE,92.141,178.0,383.8,591.7,40.6,316.0,0.264,0.257,0.867,293.0,0.4,-5.817,0.1224,-0.000066,1.173000e-08,467.33,255.24,11.95,29.16,16.0137,3096.52,-53.67,410.0,280.0,56.785,-6283.5,-5.681,4.84,7930.0


Logo, pela pesquisa, sabemos que o número dos componentes desejados no banco de dados são 242 e 286.


In [ ]:
# O comprimento da lista será o número de componentes
lista_componentes = [ 242, 286]
nc = len(lista_componentes)
nc

2

In [ ]:
# Encontrando as temperaturas de ebulição dos dois componenetes: 
T_eb_comp = dados[dados['num'].isin(lista_componentes)]['boiling_point']
T_eb_comp = T_eb_comp.tolist()
T_eb_comp

[353.3, 383.8]

In [ ]:
# Definindo as condições de temperatura e pressão do flash
# Escolhendo um valor de temperatura intermediário entre os P.E dos componentes:
T_flash = 95 + 273.15 # K
P_flash = 760.0 # mmHg

Agora iremos definir a função de Antoine:

In [ ]:
def f_Pvap_Antoine_db(Temp, i_comp, dados):
  par_array = np.array(dados[dados['num'] == i_comp][['pvap_a','pvap_b','pvap_c']])[0]
  par = {'a': par_array[0], 'b': par_array[1], 'c': par_array[2]}
  a = par['a']
  b = par['b']
  c = par['c']
  Pvap = np.exp(a - b/(Temp + c))
  return Pvap, par

Encontrando a pressão de vapor do benzeno e do tolueno:

In [ ]:
P_vap_comp = np.empty(nc)
k = 0
for i_comp in lista_componentes:
  #print(i_comp)
  print(dados[dados['num'] == i_comp]['name'])
  P_vap_comp[k], par = f_Pvap_Antoine_db(T_flash, i_comp, dados)
  k += 1
P_vap_comp

161    BENZENE
Name: name, dtype: category
Categories (465, object): ['1-BUTENE', '1-BUTYNE', '1-CHLORO-1,1-DIFLUOROETHANE',
                           '1-CHLOROBUTANE', ..., 'VINYL METHYL ETHER', 'VINYLACETYLENE', 'WATER',
                           'XENON']
440    TOLUENE
Name: name, dtype: category
Categories (465, object): ['1-BUTENE', '1-BUTYNE', '1-CHLORO-1,1-DIFLUOROETHANE',
                           '1-CHLOROBUTANE', ..., 'VINYL METHYL ETHER', 'VINYLACETYLENE', 'WATER',
                           'XENON']


array([1176.77514761,  476.86015725])

Agora, passaremos para a parte de resolução do sistema, utilizando matrizes.

Seguem as equações necessárias:

$$(1)\ \sum x_i  = x_B \ + \ x_T \ = 1 $$
$$(2)\ \sum y_i  = y_B \ + \ y_T \ = 1  $$
$$(3)\ BMG \ : \ E\ = \ V \ + \ L $$
$$(4)\ BMB \ : \ z_B \cdot E \ = \ y_B\cdot V \ + \ x_B\cdot L  $$
$$(5)\ BMT \ : \ z_T \cdot E \ = \ y_T\cdot V \ + \ x_T\cdot L  $$
$$(6)\ ELV:\ x_T \cdot P_{vap}^T(T_{flash}) = y_B \cdot P_{flash} $$
$$(7)\ ELV:\ x_B \cdot P_{vap}^B(T_{flash}) = y_T \cdot P_{flash}  $$

Primeiro iremos resolver o sistema contendo as equações: 1,2,6 e 7.

In [ ]:
#Construindo a matriz dos termos independentes:

v_rhs = np.array([1,1,0,0])
v_rhs

array([1, 1, 0, 0])

In [ ]:
# Construindo a matriz dos coeficientes das equaões 1,2,6 e 7.
A_elv = np.array([[1,1,0,0],
                  [0,0,1,1],
                  [P_vap_comp[0],0,-P_flash,0],
                  [0, P_vap_comp[1], 0, -P_flash]])
A_elv

array([[ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         1.00000000e+00],
       [ 1.17677515e+03,  0.00000000e+00, -7.60000000e+02,
         0.00000000e+00],
       [ 0.00000000e+00,  4.76860157e+02,  0.00000000e+00,
        -7.60000000e+02]])

In [ ]:
# Solução utilizando matriz inversa
sol = np.linalg.inv(A_elv) @ v_rhs
sol

array([0.40453462, 0.59546538, 0.62637669, 0.37362331])

In [ ]:
# Arrumando os resultados:
x_eq = np.empty(nc)
y_eq = np.empty(nc)
#
x_eq[0] = sol[0]
x_eq[1] = sol[1]
y_eq[0] = sol[2]
y_eq[1] = sol[3]


x_eq, y_eq

(array([0.40453462, 0.59546538]), array([0.62637669, 0.37362331]))

Agora devemos pegar as equações 3 e 4 e resolver o sistema utilizando o mesmo método de matrizes.


In [ ]:
#Definindo vazão de entrada e a composição de entrada:

E = float(50)
z_e = np.array([0.5,0.5])

In [ ]:
E

50.0

Sistma de equações 3 e 4 a serem resolvidos:


$ (3)\ V + L = E$

$ (4)\ y_B \cdot V + x_B \cdot L = z_B \cdot E$




In [ ]:
#Montando a matriz dos coeficientes independentes:
v_rhs_2 = np.array([E,float(z_e[0])*E])
v_rhs_2

array([50., 25.])

In [ ]:
#Monstando a matriz dos coeficientes:

B_elv = np.array([[1,1],
                  [y_eq[0], x_eq[0]]])
B_elv

array([[1.        , 1.        ],
       [0.62637669, 0.40453462]])

In [ ]:
# Solução utilizando matriz inversa
sol_2 = np.linalg.inv(B_elv) @ v_rhs_2
sol_2 

array([21.51651889, 28.48348111])

Resposta final:

$$V = 21.52$$
$$L = 28.48$$
$$x_B = 0.4045$$
$$x_T = 0.5955$$
$$y_B = 0.6264$$
$$y_T = 0.3736$$